Here I keep some of the old models, for reference in the paper, or to check for further improvements. Not assured that any of these do actually work.

In [2]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.9.4, build 2068644735
Copyright (C) 2014-2025 Monash University, NICTA, Data61


In [ ]:
%%minizinc
% Derived from Opponent model #4 version
% Adding periods to the model with chatgpt help
% Adding output properly formatted
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;
set of int: periods = 0..(n div 2)-1;

array[0..weeks-1, 1..n] of var 1..n: opponent;
array[0..weeks-1, 1..n] of var periods: period;  % <-- FIXED: period now depends on week

% 2 Each team plays once a week
constraint
  forall(w in 0..weeks-1) (
    all_different([opponent[w, t] | t in 1..n])
  );

% 3 Every team plays at most twice in the same period over the tournament
constraint
  forall(t in 1..n, p in periods) (
    sum(w in 0..weeks-1)( bool2int(period[w, t] = p) ) <= 2
  );

% 1 Every team plays with every other team only once  
constraint 
  forall(i, j in 1..n where i < j)(
    sum(w in 0..weeks-1)( bool2int(opponent[w, i] = j)) = 1
  );

% 4 team not playing against itself
constraint
  forall(w in 0..weeks-1, t in 1..n) (
    t != opponent[w, t]
  );

% LINKING opponent <-> period tables
constraint
  forall(w in 0..weeks-1, i, j in 1..n) (
    opponent[w, i] = j -> (opponent[w, j] = i /\ period[w, j] = period[w, i])
  );

% Possible additional symmetry breaking constraint
% 5 Break symmetry: team 1 plays teams 2,3,...,n in order
constraint forall(w in 0..weeks-1) (
    opponent[w, 1] = w + 2
);

solve satisfy;

% output [
%   "Tournament matrix by periods (rows = weeks, cols = periods):\n"
% ] ++
% [
%   "Week " ++ show(w) ++ ": " ++
%   join("  ", [
%     "[" ++ join(",", [ show(t) | t in 1..n where fix(period[w, t]) = p ]) ++ "]"
%     | p in periods
%   ]) ++ "\n"
%   | w in 0..weeks-1
% ];


Trick model

In [ ]:
%%minizinc
% Trick's model
% works for n<=32
include "alldifferent.mzn";

int: n;
set of int: teams = 1..n;
set of int: weeks = 0..n-2;
set of int: opponents = 1..n;

array[weeks, teams] of var opponents: opponent;

% 1 and 4 from opponent version at the same time
constraint 
forall(i in teams) (
    %  1 Each team plays all other teams exactly once (alldifferent over time)
    alldifferent([opponent[w, i] | w in weeks]) /\
    
    % 2 Symmetry: if i plays j at time t, then j plays i at time t
    forall(w in weeks) (
        opponent[w, opponent[w, i]] = i /\
        opponent[w, i] != i  % 3 No self-play
    )
);

constraint forall(w in weeks) (
    % 4 In each time slot, each team has exactly one opponent
    alldifferent([opponent[w, i] | i in teams]) /\
    
    % 5 Break symmetry: team 1 plays teams 2,3,...,n in order
    opponent[w, 1] = w + 2
);

solve satisfy;

Different version of STS model from the same paper down here (french professor)

In [ ]:
%%minizinc
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;
int: periods = n div 2;

array[1..n, 1..weeks] of var 1..n: opponent;
array[1..n, 1..weeks] of var 1..periods: period;

% 1. Each team plays with every other team only once
constraint
  forall(t in 1..n) (
    forall(r1, r2 in 1..weeks where r1 < r2) (
      opponent[t, r1] != opponent[t, r2]
    )
  );

% 2. Each week opponents form a proper matching
constraint
  forall(r in 1..weeks) (
    all_different([opponent[t, r] | t in 1..n])
  );

% 3. No team plays itself
constraint
  forall(t in 1..n, r in 1..weeks) (
    t != opponent[t, r]
  );

% 4. Each team plays at most twice in the same period over the tournament
constraint
  forall(t in 1..n, p in 1..periods) (
    sum([ bool2int(period[t, r] = p) | r in 1..weeks ]) <= 2
  );
  
% 5. If two teams play each other, they must be in the same period
constraint
  forall(t1 in 1..n, r in 1..weeks) (
    let {var 1..n: t2 = opponent[t1, r]} in
    period[t1, r] = period[t2, r]
  );

solve satisfy;

In [ ]:
%%minizinc
% Derived from Opponent model #4 version
% Adding periods to the model with chatgpt help
% Adding output properly formatted
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;
set of int: periods = 0..(n div 2)-1;

array[0..weeks-1, 1..n] of var 1..n: opponent;
array[periods, 1..n] of var periods: period;

% 2 Each team plays once a week
constraint
  forall(w in 0..weeks-1) (
    all_different([opponent[w, t] | t in 1..n])
  );

% 3 Every team plays at most twice in the same period over the tournament
constraint
  forall(t in 1..n, p in periods) (
    sum(w in 0..weeks-1)( bool2int(period[w, t] = p) ) <= 2
  );

% 1 Every team plays with every other team only once  
constraint 
  forall(i, j in 1..n where i < j)(
    sum(w in 0..weeks-1)( bool2int(opponent[w, i] = j)) = 1
  );

% 4 team not playing against itself
constraint
  forall(w in 0..weeks-1, t in 1..n) (
    t != opponent[w, t]
  );

% LINKING opponent <-> period tables
constraint
  forall(w in 0..weeks-1, i in 1..n) (
    let { var int: j = opponent[w, i] } in
      (opponent[w, j] = i /\ period[w, j] = period[w, i])
  );

% Possible additional symmetry breaking constraint
% 5 Break symmetry: team 1 plays teams 2,3,...,n in order
constraint forall(w in 0..weeks-1) (
    opponent[w, 1] = w + 2
);

solve satisfy;

% output [
%   "Opponents per week:\n"
% ] ++
% [
%   "Week \(w): " ++ show([opponent[w, t] | t in 1..n]) ++ "\n"
%   | w in 0..weeks-1
% ] ++
% [
%   "\nPeriods per week:\n"
% ] ++
% [
%   "Week \(w): " ++ show([period[w, t] | t in 1..n]) ++ "\n"
%   | w in 0..weeks-1
% ];


In [ ]:
%%minizinc
% Derived from Opponent model #2 version
% Adding periods to the model
% works for n <=80 (and probably even more who knows)
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;
set of int: periods = 0..(n div 2)-1;

%array[1..n, 1..weeks] of var 1..n: opponent;
array[0..weeks-1, 1..n] of var 1..n: opponent;
array[periods, 1..n] of var 1..n: period;

% 2 Each team plays once a week
constraint
  forall(w in 0..weeks-1) (
    all_different([opponent[w, t] | t in 1..n])
  );

% 3 Every team plays at most twice in the same period over the tournament
constraint
  forall(t in 1..n, p in periods) (
    sum(i in 1..n)( bool2int(period[p, i] = t) ) <= 2
  );

% 1 Every team plays with every other team only once  
constraint 
  forall(w in 0..weeks-1) (
    alldifferent([opponent[w, t] | t in 1..n])
  );
 
% 4 In each time slot, each team has exactly one opponent
constraint forall(w in 0..weeks-1) (
    alldifferent([opponent[w, i] | i in 1..n])
    );

%4 team not playing against itself
constraint
  forall(w in 0..weeks-1, t in 1..n) (
    t != opponent[w, t]
  );
  
% BREAKING SYMMETRIES

% Symmetry: if i plays j at time t, then j plays i at time t
% TODO adding constraint to link table for periods with same rule and same teams!
constraint 
forall(w in 0..weeks-1) (
  forall(p in periods)(
    forall(i in 1..n) ( 
      opponent[w, opponent[w, i]] = i /\ 
      period[p, period[p, i]] = i /\
      period[p, i] == opponent [w, i]
      )
    )
  );


%Possible additional symmetry breaking constraint
% 5 Break symmetry: team 1 plays teams 2,3,...,n in order
constraint forall(w in 0..weeks-1) (
    opponent[w, 1] = w + 2
    );

solve satisfy;

% output [
%   "Opponents:\n"
% ] ++
% [
%   "Weeks \(w): " ++ show([opponent[w, t] | t in 1..n]) ++ "\n"
%   | w in 1..weeks
% ];


In [ ]:
%%minizinc
% Derived from Opponent model #1 version
% works for n <=80 (and probably even more who knows)
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;

%array[1..n, 1..weeks] of var 1..n: opponent;
array[0..weeks-1, 1..n] of var 1..n: opponent;

% 2 Each team plays once a week
constraint
  forall(w in 0..weeks-1) (
    all_different([opponent[w, t] | t in 1..n])
  );

% 1 Every team plays with every other team only once  
constraint 
  forall(w in 0..weeks-1) (
    alldifferent([opponent[w, t] | t in 1..n])
  );
 
%also this one? 
% constraint forall(w in weeks) (
%     % 4 In each time slot, each team has exactly one opponent
%     alldifferent([opponent[w, i] | i in teams])

%4 team not playing against itself
constraint
  forall(w in 0..weeks-1, t in 1..n) (
    t != opponent[w, t]
  );
  
% BREAKING SYMMETRIES

% Symmetry: if i plays j at time t, then j plays i at time t
% TODO adding constraint to link table for periods with same rule and same teams!
constraint 
forall(w in 0..weeks-1) (
  forall(i in 1..n) (
        opponent[w, opponent[w, i]] = i 
        )
    );

%Possible additional symmetry breaking constraint
% 5 Break symmetry: team 1 plays teams 2,3,...,n in order
% constraint forall(w in weeks) (
%     opponent[w, 1] = w + 2
%     );

solve satisfy;

% output [
%   "Opponents:\n"
% ] ++
% [
%   "Weeks \(w): " ++ show([opponent[w, t] | t in 1..n]) ++ "\n"
%   | w in 1..weeks
% ];


In [ ]:
%%minizinc
% Opponent model 
% works for n <=38
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;

array[1..n, 1..weeks] of var 1..n: opponent;

% 2 Each team plays once a week
constraint
  forall(r in 1..weeks) (
    all_different([opponent[t, r] | t in 1..n])
  );

% 1 Every team plays with every other team only once  
constraint
forall(t in 1..n) (
  forall(r1, r2 in 1..weeks where r1 < r2) (
    opponent[t, r1] != opponent[t, r2]
  )
);

%4 team not playing against itself
constraint
  forall(t in 1..n, r in 1..weeks) (
    t != opponent[t, r]
  );

solve satisfy;

output [
  "Opponents:\n"
] ++
[
  "Team \(t): " ++ show([opponent[t, r] | r in 1..weeks]) ++ "\n"
  | t in 1..n
];


In [ ]:
%%minizinc
include "alldifferent.mzn";

int: n;  % number of teams
int: weeks = n - 1;
int: periods = n div 2;

array[1..n, 1..weeks] of var 1..n: opponent;
array[1..n, 1..weeks] of var 1..periods: period;

% 1. Each team plays with every other team only once
constraint
  forall(t in 1..n) (
    forall(r1, r2 in 1..weeks where r1 < r2) (
      opponent[t, r1] != opponent[t, r2]
    )
  );

% 2. Each week opponents form a proper matching
constraint
  forall(r in 1..weeks) (
    all_different([opponent[t, r] | t in 1..n])
  );

% 3. No team plays itself
constraint
  forall(t in 1..n, r in 1..weeks) (
    t != opponent[t, r]
  );

% 4. Each team plays at most twice in the same period
constraint
  forall(t in 1..n, p in 1..periods) (
    sum([ bool2int(period[t, r] = p) | r in 1..weeks ]) <= 2
  );

% 5. If two teams play each other, they must be in the same period
constraint
  forall(t1 in 1..n, r in 1..weeks) (
    let {var 1..n: t2 = opponent[t1, r]} in
    period[t1, r] = period[t2, r]
  );

solve satisfy;

% ---------- OUTPUT SECTION ----------

%RUNNING BUT WRONG FORMATTING
output [
  "\nTeam schedule:\n"
] ++
[
  "Team " ++ show(t) ++ ": " ++ 
  join(", ", [
    "W" ++ show(w) ++ ": vs " ++ show(opponent[t, w]) ++ " (P" ++ show(period[t, w]) ++ ")"
    | w in 1..weeks
  ]) ++ "\n"
  | t in 1..n
];


%RUNNING BUT WRONG RESULT
% output
% ["["]++
% [
%   "\n[" ++
%   join("", [
%     let {
%       set of int: teams_in_period = { t | t in 1..n where fix(period[t, r]) == p /\ t < fix(opponent[t, r]) }
%     } in
%     join("", [
%       "[" ++ show(t) ++ ", " ++ show(fix(opponent[t, r])) ++ "], "
%       | t in teams_in_period
%     ])
%     | r in 1..weeks
%   ]) ++ "]"
%   | p in 1..periods
% ]++
% ["\n]"];

In [ ]:
# not correct translation from opl
%%minizinc
% MODEL TAKEN FROM OPL PREVIEW translated with chatgpt
int: nbTeams;
set of int: Teams = 1..nbTeams;
set of int: Weeks = 1..nbTeams-1;
set of int: EWeeks = 1..nbTeams;
set of int: Periods = 1..nbTeams/2;
set of int: Games = 1..nbTeams*nbTeams;
set of int: slots = 1..2; %is this correct?

%occur
%values

array[Periods, EWeeks, Slots] of var int Teams: team;
array[Periods, Weeks] of var int Games: game;

%predicate link

constraint forall(w in EWeeks)(
    alldifferent(team[p,w,s] | p in Periods, s in Slots)
);

constraint
  alldifferent(game);
  
constraint forall(p in Periods)
  %distribute??
  
constraint link

solve satisfy;



In [ ]:
%%minizinc
% Use this editor as a MiniZinc scratch book
% Parameters
include "alldifferent.mzn";
include "distribute.mzn";

int: nbTeams;
set of int: Teams = 1..nbTeams;
set of int: Weeks = 1..nbTeams-1;
set of int: EWeeks = 1..nbTeams;
set of int: Periods = 1..nbTeams div 2;
set of int: Games = 1..nbTeams*nbTeams;

array[Teams] of int: occur = [2 | t in Teams];
array[Teams] of int: values = Teams;

% Decision variables
array[Periods, EWeeks, 1..2] of var Teams: team;
array[Periods, Weeks] of var Games: game;

% Link constraint: game encoding
predicate link(var int: f, var int: s, var int: g) =
  g = (f - 1) * nbTeams + s;

% Constraints-----------------------------------------------------

% All teams play in each extended week (alldifferent constraint)
constraint forall(w in EWeeks) (
  alldifferent([team[p,w,s] | p in Periods, s in 1..2])
);

%link
constraint forall(p in Periods, w in Weeks)(
  link(team[p, w, 1], team[p, w, 2], game[p,w])
);

% All games are different
% Flatten the game array for alldifferent
constraint alldifferent([game[p,w] | p in Periods, w in Weeks]);
constraint forall(p in Periods, w in Weeks)(
  team[p, w, 1] < team[p, w, 2]
);

% Each team appears exactly twice for each period (distribute constraint)
constraint forall(p in Periods) (
  distribute(occur, values, [team[p,w,s] | w in EWeeks, s in 1..2])
);

% Search annotation
solve satisfy;

output ["["++
  join(",\n ",[ 
    "[" ++ join(",", ["[\(team[p,w,1]),\(team[p,w,2])]" | w in Weeks]) ++ "]"
    | p in Periods
  ])++"]"
];

In [ ]:
# last tried non-official version
%%minizinc
%%writefile "cp_model.mzn"
include "alldifferent.mzn";

%% INSTANCE VARIABLES

int: n;
int: weeks = n - 1;
int: periods = n div 2;
int: slots = 2;

set of int: Weeks =  1..weeks;
set of int: Periods = 1..periods;

%% DECISION VARIABLES

array[Periods, Weeks, 1..slots] of var 1..n: game;

% [0] n is even
constraint n mod 2 = 0;

% [1] Every team plays with every other team only once
constraint alldifferent([game[p,w,1]*n + game[p,w,2] | p in Periods, w in Weeks]);
% constraint
%     global_cardinality_low_up(
%         [game[p,w,1]*n + game[p,w,2] | p in Periods, w in Weeks],
%         [i | i in 1..n*n], % these are the n teams otherwise some teams play more than once
%         [0 | i in 1..n*n],
%         [1 | i in 1..n*n]   % 1 appearance x week
% );

% [2] Every team plays once a week
% constraint 
%   forall(w in Weeks)(
%     alldifferent([game[p,w,s] | p in 1..periods, s in 1..slots])
% );
constraint forall(w in Weeks) (
    global_cardinality(
        [game[p,w,s] | p in Periods, s in 1..slots],
        [i | i in 1..n], % these are the n teams otherwise some teams play more than once
        [1 | i in 1..n]   % 1 appearance x week
    )
);

% [3] Every team plays at most twice in the same period over the tournament 
constraint forall(p in Periods) (
    global_cardinality_low_up(
        [game[p,w,s] | w in Weeks, s in 1..slots],
        [i | i in 1..n], % these are the n teams otherwise some teams play more than once
        [1 | i in 1..n],  % lower bound: 0 appearances
        [2 | i in 1..n]   % upper bound: 2 appearances
    )
);

% [4] implied every game must be between different teams + symmetry breaking ordering of teams in a game
constraint forall(w in Weeks, p in Periods) (  game[p,w,1] < game[p,w,2]);

%% SYMMETRY BREAKING CONSTRAINTS

% [5]
constraint game[1, 1, 1] = 1;

% [6]
constraint forall(w in 2..weeks-1) (  game[1,w, 1] < game[1,w+1, 1]);

% SEARCHING STRATEGY 
% restart + randomization to optimiza search

% restart sequence type = luby
% base for geometric restart sequence 1.5
solve :: seq_search([
    int_search(
        [game[p,w,s] | w in Weeks, p in Periods, s in 1..slots],
         most_constrained,   % variable with smallest domain and most constraint impact
         indomain_random,    % value selection: random
         complete
    )
]) satisfy;

% OUTPUT FORMATTING

output ["["++
  join(",",[ 
    "[" ++ join(",", [ show([game[i,j,k] | k in 1..2]) | j in 1..weeks ]) ++ "]"
    | i in 1..periods
  ]) ++ "]"
];